In [2]:
import zipfile
local_path='./archive.zip'
zip_ref=zipfile.ZipFile(local_path,'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
import os
train_dir=os.path.join('./Covid19-dataset/train')
test_dir=os.path.join('./Covid19-dataset/test')

In [4]:
train_names=os.listdir(train_dir)
print(train_names)
test_names=os.listdir(test_dir)
print(test_names)

['Covid', 'Normal', 'Viral Pneumonia']
['Covid', 'Normal', 'Viral Pneumonia']


In [7]:
train_Covid_names=os.listdir('./Covid19-dataset/train/Covid')
print(train_Covid_names[0:5])

['01.jpeg', '010.png', '012.jpeg', '015.jpg', '019.png']


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen=ImageDataGenerator(rescale=1.0/255.0,
                            height_shift_range=0.2,
                            width_shift_range=0.2,
                            rotation_range=40,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')
test_gen=ImageDataGenerator(rescale=1.0/255.0)

In [16]:
train_gen=ImageDataGenerator(rescale=1.0/255.0)
test_gen=ImageDataGenerator(rescale=1.0/255.0)

In [17]:
train_generator=train_gen.flow_from_directory(train_dir,
                                                  target_size=(300,300),
                                                  batch_size=32,
                                                  class_mode='categorical')
test_generator=test_gen.flow_from_directory(train_dir,
                                                  target_size=(300,300),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 251 images belonging to 3 classes.
Found 251 images belonging to 3 classes.


In [13]:
model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(512,activation='relu'),
                                 tf.keras.layers.Dense(3,activation='sigmoid')])
model.build()

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 64)        0

In [18]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             metrics=['accuracy'])
history=model.fit(train_generator, epochs=20, validation_data=test_generator)

Epoch 1/20
8/8 [==============================] - 15s 2s/step - loss: 0.5605 - accuracy: 0.7689 - val_loss: 0.4653 - val_accuracy: 0.7729
Epoch 2/20
8/8 [==============================] - 15s 2s/step - loss: 0.4224 - accuracy: 0.8247 - val_loss: 0.2844 - val_accuracy: 0.9163
Epoch 3/20
8/8 [==============================] - 15s 2s/step - loss: 0.3019 - accuracy: 0.8805 - val_loss: 0.2316 - val_accuracy: 0.9203
Epoch 4/20
8/8 [==============================] - 14s 2s/step - loss: 0.2281 - accuracy: 0.9203 - val_loss: 0.1794 - val_accuracy: 0.9323
Epoch 5/20
8/8 [==============================] - 13s 2s/step - loss: 0.1727 - accuracy: 0.9363 - val_loss: 0.1405 - val_accuracy: 0.9482
Epoch 6/20
8/8 [==============================] - 13s 2s/step - loss: 0.1569 - accuracy: 0.9363 - val_loss: 0.1299 - val_accuracy: 0.9442
Epoch 7/20
8/8 [==============================] - 13s 2s/step - loss: 0.1495 - accuracy: 0.9363 - val_loss: 0.1169 - val_accuracy: 0.9562
Epoch 8/20
8/8 [==================

In [19]:
model.save('model.h5')

C:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
